In [2]:
import os
os.chdir('..')
from src.construct import util
import pandas as pd
idx = pd.IndexSlice

In [34]:
annual_energy_balance = util.read_tdf('build/annual_energy_balances.csv')

In [20]:
# in terms of energy, from annual energy balance (all waste streams)
waste_consumption_2018 = annual_energy_balance.xs(('TI_EHG_E', 2018), level=('cat_code', 'year')).loc[idx[['W6210', 'W6100_6220'], :, :]].sum()

In [23]:
# In terms of thousand tonnes, 2018, see https://appsso.eurostat.ec.europa.eu/nui/show.do?dataset=env_wasmun&lang=en
waste_consumption_tonnes =  (
    2022  + 208  + 616  + 2302  + 15936  + 221  + 1243  + 83  + 2898  + 12370  + 1  + 5756  + 15  + 163  + 162  + 501  + 0  + 3762  + 1977  + 3013  + 936  + 241  + 105  + 187  + 1733  + 2362  + 12046  + 2006  + 2856  + 0  + 63  + 0  + 0 
) 
waste_consumption_tj = waste_consumption_tonnes * 10 # 8-12MJ/kg, see https://www.ieabioenergy.com/wp-content/uploads/2013/10/40_IEAPositionPaperMSW.pdf

In [46]:
# effective energy content of waste, when combining annual energy balance data and waste consumed in tonnes
# This is in the range of 8-12MJ/kg given by the IEA
print(f"Energy content of waste: {waste_consumption_2018 / waste_consumption_tonnes:.2f} TJ/ktonnes (MJ/kg)")

Energy content of waste: 11.14 TJ/ktonnes (MJ/kg)


In [44]:
# total available energy, including Switzerland
print(f"Total available energy from waste: {(waste_consumption_2018 + 13532 / 0.27778) * 0.27778 / 1e3:.2f} TWh")

Total available energy from waste: 248.12 TWh


In [19]:
swiss_waste = pd.read_excel('data/automatic/ch_annual_energy_balances.xlsx', sheet_name='T27', skiprows=5, index_col=0, header=[0, 1], skipfooter=8)[("Consommation d'énergie (GWh)", "Ordures")]
swiss_waste

In [33]:
swiss_waste[("Consommation d'énergie (GWh)", "Ordures")].to_frame('bar').rename_axis(index='year', columns='foo').stack()

year  foo
1990  bar     7495
1991  bar     7550
1992  bar     7438
1993  bar     7450
1994  bar     7556
1995  bar     7431
1996  bar     7346
1997  bar     7649
1998  bar     8081
1999  bar     8735
2000  bar     9444
2001  bar     9987
2002  bar    10266
2003  bar    10140
2004  bar    10304
2005  bar    10800
2006  bar    11910
2007  bar    11910
2008  bar    11792
2009  bar    11699
2010  bar    12111
2011  bar    12285
2012  bar    12595
2013  bar    12164
2014  bar    12389
2015  bar    12796
2016  bar    13263
2017  bar    13333
2018  bar    13445
2019  bar    13532
dtype: int64

In [38]:
annual_energy_balance.loc[idx["TI_EHG_E", ["W6210", "W6100_6220"], "TJ", :, :]].sum(level=['country', 'year']).rename_axis(index={'country': 'country_code'})

country_code  year  
AL            1990.0        0.0
              1991.0        0.0
              1992.0        0.0
              1993.0        0.0
              1994.0        0.0
                         ...   
UK            2014.0    50721.0
              2015.0    63212.0
              2016.0    83826.0
              2017.0    79042.0
              2018.0    90985.0
Name: 0, Length: 947, dtype: float64

In [43]:
(
    annual_energy_balance.loc[idx["TI_EHG_E", ["W6210", "W6100_6220"], "TJ", :, :]]
    .sum(level=['country', 'year', 'unit'])
    .apply(util.tj_to_twh)
    .rename_axis(index={"country": "country_code"})
    .rename(util.get_alpha3, level='country_code')
    .rename({"TJ": "twh"}, level='unit')
).xs((2018, "twh"), level=('year', "unit"))['ALB']

0.0